In [83]:
#mounting the drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [84]:
#importing header files
import pandas as pd
import numpy as np
import os
import pickle
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [85]:
############################################### PART A #############################################
#convert text to lowercase
def convert_lower_case(text):
  return np.char.lower(text)

#remove underscore
def rem_underscore(text):
  t=str(text)
  return t.replace("_","")

#Remove stopwords from text
def remove_stop_words(text):  
    stop_words = stopwords.words('english')
    token = word_tokenize(str(text))
    text2 = ""
    for t in token:
      if t not in stop_words:
        text2 = text2+" "+t
    return np.char.strip(text2)

#Remove punctuation marks from text
def remove_punctuation(text):   
    symbols = "/:;!\"#+-.&()*^_`<=>?@[\                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               ]$%{|}~\n"
    for i in range(len(symbols)):
        text = np.char.replace(text, symbols[i], ' ')
        text = np.char.replace(text, "  ", " ")
    text = np.char.replace(text, ',', ' ')
    text = np.char.replace(text, "'", "")
    return text

#Remove blank space
def remove_blank_space(text):
  t=str(text)
  return " ".join(t.split())

#Remove single characters
def remove_single_char(text):
    words = word_tokenize(str(text))
    text2 = ""
    for w in words:
        if len(w) > 1:
            text2 = text2 + " " + w
    return np.char.strip(text2)  

#convert numbers to text
def convert_number_to_text(text):
    text = np.char.replace(text, "0", " zero ")     #replace 0 by zero in text.
    text = np.char.replace(text, "1", " one ")      #replace 1 by one in text.
    text = np.char.replace(text, "2", " two ")      #replace 2 by two in textd.
    text = np.char.replace(text, "3", " three ")    #replace 3 by three in text.
    text = np.char.replace(text, "4", " four ")     #replace 4 by four in text.
    text = np.char.replace(text, "5", " five ")     #replace 5 by five in text.
    text = np.char.replace(text, "6", " six ")      #replace 6 by six in text.
    text = np.char.replace(text, "7", " seven ")    #replace 7 by seven in text.
    text = np.char.replace(text, "8", " eight ")    #replace 8 by eight in text.
    text = np.char.replace(text, "9", " nine ")     #replace 9 by nine in text.
    return text

#performing lemmatization on data
def lem(text):
  lemmatizer = WordNetLemmatizer()
  tokens = word_tokenize(str(text))
  text2 = ""
  for w in tokens:
      text2 = text2 + " " + lemmatizer.lemmatize(w)
  return np.char.strip(text2)

#removing non-ascii characters
def remove_non_ascii(text):
  t=str(text)
  return t.encode("ascii","ignore").decode("utf-8","ignore")

#perform preprocessing
def preprocess_data(text):  
  text = convert_lower_case(text)
  text = rem_underscore(text)
  text = convert_number_to_text(text)
  text = remove_punctuation(text)
  text = remove_stop_words(text)
  text = remove_blank_space(text)
  text = remove_single_char(text)
  text = remove_non_ascii(text)
  text = lem(text)
  return text

In [87]:
################################################## PART B ############################################
def create_inverted_index(paths):     #function create inverted index.
  store_filename = []                 #list for storing all file names
  docID = 0                           #counter for documents
  tii2=dict()                         # dictionary for storing index with word as key and position as values

  #loop for calculating index for each file 
  for path in paths: 
    file = open(path, 'r', encoding= 'ISO-8859-1')    
    text = " ".join(file.read().split())    # for removing extra spaces
    file.close()
    print(docID)
    preprocessed_data = preprocess_data(text)
    tokens = word_tokenize(str(preprocessed_data))      #tokenize the preprocessed text and convert into tokens
      
    #loop for storing data in dictionary
    for t in tokens:
      if t not in tii2.keys():
        tii2[t] = list()
        tii2[t].append(docID)
      else:
        if docID not in tii2[t]:
          tii2[t].append(docID)

    filename = os.path.basename(path)        #extract last filename from  the path
    store_filename.append([filename])        #append into list
    docID += 1
    
  store_filename = pd.DataFrame(store_filename)     #convert filenames list of list into dataframe

  # save the inverted index and filenames in pickle file so that can we build futher
  with open('fn.pickle', 'wb') as handle:
    pickle.dump(tii2, handle, protocol=pickle.HIGHEST_PROTOCOL)

  store_filename.to_pickle("sf")

In [79]:
#################################################### PART C #######################################################
def not_operation(var):                      #function to perform not operation
  v = set(range(len(paths)))
  return list(v.difference(var))

def or_operation(var1,var2):                 #function to perform or operation
  n = len(var1)                              #store length of var1 and var2 into n and m.
  m = len(var2)
  Result = []
  comparisons = 0
  i = 0
  j = 0

  while i<n and j<m:            #perform union in two set X and Y and count number of comparision.
    if var1[i]==var2[j]:
      Result.append(var1[i])
      i=i+1
      j=j+1
      comparisons=comparisons+1
    elif var1[i]<var2[j]:
      Result.append(var1[i])
      i=i+1
      comparisons=comparisons+1
    else:
      Result.append(var2[j])
      j=j+1
      comparisons=comparisons+1 

  while i<n:                      #when only var1 set left then append into result without any comparision.
    Result.append(var1[i])
    i=i+1

  while j<m:                      #when only var2 set left then append into result without any comparision.
    Result.append(var2[j])
    j=j+1  

  return Result, comparisons           #return union of var1 and var2 and number of comparision.

def and_operation(var1,var2):        #function to perform and operation
  n = len(var1)                      #store length of var1 and var2 into n and m.
  m = len(var2)
  Result = []
  comparisons =0
  i = 0
  j = 0

  while i<n and j<m:           #perform union in two set var1 and var2 and count number of comparision.
    if var1[i]==var2[j]:
      Result.append(var1[i])
      i=i+1
      j=j+1
      comparisons=comparisons+1
    elif var1[i]<var2[j]:
      i=i+1
      comparisons=comparisons+1
    else:
      j=j+1
      comparisons=comparisons+1
  return Result, comparisons     #return intersection of var1 and var2 and number of comparision.

def or_not_operation(var1, var2):                    #function to perform or Not operation
    var2 = not_operation(var2)                       #find not of var2.
    Result, comparisons = or_operation(var1, var2)   #after finding not of var2 then perform OR of var1 and var2 and find no of comparision.
    return Result, comparisons                       #return the result after var1 or Not var2 and number of comparision.
  
def and_not_operation(var1, var2):                   #function to perform and not operation
    var2 = not_operation(var2)                       #find not of var2.
    Result, comparisons = and_operation(var1, var2)  #after finding not of Y then perform and of var1 and var2 and find number of comparision.
    return Result, comparisons                       #return the result after var1 or Not var2 and number of comparision.

In [80]:
def cal_result(Query_docIds):
  op_pos = 0                        #iterator for positions
  Total_comp = 0                    #to store total comaprisons
  X = Query_docIds[0]
  for operand in Query_docIds[1:]:
    Y = operand

    operator = operations[op_pos]
    op_pos = op_pos+1

    if(operator == 'OR'):      #if operator is OR then call Function_or and find resultant list and number of comparision.
      X, No_of_compare = or_operation(X, Y)
      Total_comp = Total_comp + No_of_compare

    elif(operator == 'AND'):       #if operator is AND then call Function_or and find resultant list and number of comparision.
      X, No_of_compare = and_operation(X, Y)
      Total_comp = Total_comp + No_of_compare

    elif(operator == 'OR NOT'):       #if operator is OR NOT then call Function_or and find resultant list and number of comparision.
      X, No_of_compare = or_not_operation(X, Y)
      Total_comp = Total_comp + No_of_compare

    elif(operator == 'AND NOT'):      #if operator is AND NOT then call Function_or and find resultant list and number of comparision.
      X, No_of_compare = and_not_operation(X, Y)
      Total_comp = Total_comp + No_of_compare

    else:
      print("You Entered Wrong Operations................!!!")

  print("\n")
  print("Number of documents matched: ", len(X))
  print("No. of comparisons required: ",Total_comp)  

  # print the document names retrieved
  name_list = []
  for i in X:
    name_list.append(store_filename.iloc[i][0])
  print("Documents retreived are:-")
  print(name_list)

In [ ]:
if __name__ == "__main__":

  files_path = "/content/drive/MyDrive/Humor,Hist,Media,Food"

  # find all files those are stored into files_path and store into path.
  paths = []
  for (dirpath, dirnames, filenames) in os.walk(str(files_path)):
    for i in filenames:
       paths.append(str(dirpath)+str("/")+i)
  
  create_inverted_index(paths)    #for creating index
  store_filename = []             #for storing filenames

  # read the index and filename 
  with open('fn.pickle', 'rb') as handle:
    inverted_index = pickle.load(handle)
  store_filename=pd.read_pickle("sf")

  # Our input query
  N = int(input("Enter Number of Queries you want to test:-"))  
  while N>0:
    Query = input("Enter the query:-")
    operations  = input("Enter operations with comma between them:-")
    operations=operations.upper()
    operations=operations.split(",")
    operations = [i.strip() for i in operations]

  # Query Pre-processing
    pre_processed_query = preprocess_data(Query)
    pre_processed_query = pre_processed_query.flatten()
    pre_processed_query = np.char.split(pre_processed_query[0])
    pre_processed_query = pre_processed_query.tolist()

  #add word documents id to query document ids in sorted order
    Query_docIds = []

    for r in pre_processed_query:
      if r not in inverted_index.keys():
        print(r+" is not in any document. Please enter valid query")
        break
      word_docIds = inverted_index[r]
      Query_docIds.append(sorted(word_docIds))

    #check if processed query documents are 1 less than number of operations 
    if(len(Query_docIds)-1 == len(operations)):
      cal_result(Query_docIds)  
    else:
      print("Wrong Input")
    N-=1